In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cpu)
    Python  3.9.13 (you have 3.9.18)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
from datasets import load_dataset
import pandas as pd
import json 

dataset = load_dataset("PatronusAI/financebench")
df = dataset['train'].to_pandas()

In [9]:
finetuning_data = pd.read_csv('train.csv')

In [14]:
finetuning_data['text'][0]

'You are a financial chatbot trained to answer questions based on the information provided in 10-K documents. Your responses should be directly sourced from the content of these documents. When asked a question, ensure that your answer is explicitly supported by the text in the 10-K filing, and do not include any external information, interpretations, or assumptions not clearly stated in the document. If a question pertains to financial data or analysis that is not explicitly covered in the 10-K filing provided, respond by stating that the information is not available in the document. Your primary focus should be on accuracy, specificity, and adherence to the information in 10-K documents, particularly regarding financial statements, company performance, and market positions.\n### Instruction:\nWhat is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.\n### Input: \nTable of Content

In [18]:
import pandas as pd
import re

# Define the reformat_text function
def reformat_text(text):
    system_prompt_pattern = r"(.*?)### Instruction:"
    user_prompt_pattern = r"### Instruction:(.*?)### Input:"
    input_pattern = r"### Input:(.*?)### Response:"
    model_answer_pattern = r"### Response: \n(.*)"

    system_prompt = re.search(system_prompt_pattern, text, re.DOTALL).group(1).strip()
    user_prompt = re.search(user_prompt_pattern, text, re.DOTALL).group(1).strip()
    input_section = re.search(input_pattern, text, re.DOTALL).group(1).strip()
    model_answer = re.search(model_answer_pattern, text, re.DOTALL).group(1).strip()

    return f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt} {input_section} [/INST] {model_answer} </s>"

# Example DataFrame creation (replace this with your DataFrame loading code)

# Apply the function to the entire 'text' column
finetuning_data['formatted_text'] = finetuning_data['text'].apply(reformat_text)

# Display the DataFrame to verify the results
finetuning_data['formatted_text'][0]


'<s>[INST] <<SYS>>\nYou are a financial chatbot trained to answer questions based on the information provided in 10-K documents. Your responses should be directly sourced from the content of these documents. When asked a question, ensure that your answer is explicitly supported by the text in the 10-K filing, and do not include any external information, interpretations, or assumptions not clearly stated in the document. If a question pertains to financial data or analysis that is not explicitly covered in the 10-K filing provided, respond by stating that the information is not available in the document. Your primary focus should be on accuracy, specificity, and adherence to the information in 10-K documents, particularly regarding financial statements, company performance, and market positions.\n<</SYS>>\n\nWhat is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement. Table of Contents

In [20]:
finetuning_data = finetuning_data[['formatted_text']]
finetuning_data.rename(columns = {'formatted_text': 'text'}, inplace = True)
finetuning_data

,text
0,<s>[INST] <<SYS>>\nYou are a financial chatbot...
1,<s>[INST] <<SYS>>\nYou are a financial chatbot...
2,<s>[INST] <<SYS>>\nYou are a financial chatbot...
3,<s>[INST] <<SYS>>\nYou are a financial chatbot...
4,<s>[INST] <<SYS>>\nYou are a financial chatbot...
...,...
145,<s>[INST] <<SYS>>\nYou are a financial chatbot...
146,<s>[INST] <<SYS>>\nYou are a financial chatbot...
147,<s>[INST] <<SYS>>\nYou are a financial chatbot...
148,<s>[INST] <<SYS>>\nYou are a financial chatbot...


In [22]:
finetuning_data.to_csv("train2.csv", index = False)

In [26]:
# The model that you want to train from the Hugging Face hub
model_name="meta-llama/Llama-2-7b-chat-hf"


# Fine-tuned model name
new_model = "finetuned_llama-2_benchmarking"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [28]:
# Load dataset (you can process it here)
file_path = "C:/Spurthi/Anote_Winternship/Q-A/LLAMA-2/train2.csv"

# Load your dataset from a local file
dataset = load_dataset('csv', data_files=file_path, split='train')

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)



In [30]:

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

RuntimeError: No GPU found. A GPU is needed for quantization.

## Trying to alter it so that it works on a CPU

In [31]:
# Load dataset (you can process it here)
# Load dataset (you can process it here)
file_path = "C:/Spurthi/Anote_Winternship/Q-A/LLAMA-2/train2.csv"

# Load your dataset from a local file
dataset = load_dataset('csv', data_files=file_path, split='train')

# Load tokenizer and model (without QLoRA configuration for GPU)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(model_name)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters (adjusted for CPU)
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=False,  # Set to False for CPU
    bf16=False,  # Set to False for CPU
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\spurt\.conda\envs\Anote\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\trl\trainer\sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


AttributeError: 'NoneType' object has no attribute 'cget_managed_ptr'